# Lights model with PBC2 dataset

In [1]:
# Library setup
%reset -f
%matplotlib inline
import os
os.environ['R_HOME'] = "/Library/Frameworks/R.framework/Versions/4.0/Resources"
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from lifelines.utils import concordance_index as c_index_score
from lights.inference import prox_QNMCEM
from lights.base.utils import visualize_vect_learning, plot_history
import numpy as np
from sklearn.preprocessing import StandardScaler
from lifelines import KaplanMeierFitter
import matplotlib.pyplot as plt
import seaborn as sns
from lifelines.utils import concordance_index as c_index_score
from prettytable import PrettyTable
from time import time
from lights.competing_methods.all_model import load_data, extract_lights_feat, extract_R_feat
from matplotlib import rc
rc('text', usetex=True)
%matplotlib inline

def printmd(string):
    display(Markdown(string))
    
import rpy2.robjects as robjects
import warnings
%load_ext rpy2.ipython

## PBC_Seq

In [2]:
t = PrettyTable(['Algos', 'C_index', 'time'])
test_size = .2
simu = False
data, data_lights, time_dep_feat, time_indep_feat = load_data(simu)
id_list = data_lights["id"]
nb_test_sample = int(test_size * len(id_list))
id_test = np.random.choice(id_list, size=nb_test_sample, replace=False)
data_lights_train = data_lights[data_lights.id.isin(id_test)]
data_lights_test = data_lights[data_lights.id.isin(id_test)]
X_lights_train, Y_lights_train, T_train, delta_train = \
    extract_lights_feat(data_lights_train, time_indep_feat, time_dep_feat)
X_lights_test, Y_lights_test, T_test, delta_test = \
    extract_lights_feat(data_lights_test, time_indep_feat, time_dep_feat)

data_train = data[data.id.isin(id_test)]
data_test = data[data.id.isin(id_test)]
data_R_train, T_R_train, delta_R_train = extract_R_feat(data_train)
data_R_test, T_R_test, delta_R_test = extract_R_feat(data_test)

R[write to console]: Le chargement a nécessité le package : nlme

R[write to console]: Le chargement a nécessité le package : survival

R[write to console]: Le chargement a nécessité le package : doParallel

R[write to console]: Le chargement a nécessité le package : foreach

R[write to console]: Le chargement a nécessité le package : iterators

R[write to console]: Le chargement a nécessité le package : parallel

R[write to console]: Le chargement a nécessité le package : rstan

R[write to console]: Le chargement a nécessité le package : StanHeaders

R[write to console]: Le chargement a nécessité le package : ggplot2

R[write to console]: rstan (Version 2.21.2, GitRev: 2e1f913d3ca3)

R[write to console]: For execution on a local, multicore CPU with excess RAM we recommend calling
options(mc.cores = parallel::detectCores()).
To avoid recompilation of unchanged Stan programs, we recommend calling
rstan_options(auto_write = TRUE)



In [4]:
# The penalized time-dependent Cox model.
robjects.r.source(os.getcwd() + "/lights/competing_methods/CoxNet.R")
X_R_train = robjects.r["Cox_get_long_feat"](data_R_train, time_dep_feat)
X_R_test = robjects.r["Cox_get_long_feat"](data_R_test, time_dep_feat)
best_lambda = robjects.r["Cox_cross_val"](X_R_train, T_R_train, delta_R_train)
start = time()
trained_CoxPH = robjects.r["Cox_fit"](X_R_train, T_R_train,
                                      delta_R_train, best_lambda)
Cox_pred = robjects.r["Cox_score"](trained_CoxPH, X_R_test)
Cox_marker = np.array(Cox_pred[:])
Cox_c_index = c_index_score(T_test, Cox_marker, delta_test)
Cox_c_index = max(Cox_c_index, 1 - Cox_c_index)
Cox_exe_time = time() - start

Be patient, hlme is running ... 
The program took 0.05 seconds 
Be patient, hlme is running ... 
The program took 0.04 seconds 
Be patient, hlme is running ... 
The program took 0.06 seconds 
Be patient, hlme is running ... 
The program took 0.04 seconds 
Be patient, hlme is running ... 
The program took 0.04 seconds 
Be patient, hlme is running ... 
The program took 0.06 seconds 


In [5]:
# Multivariate joint latent class model.
start = time()
robjects.r.source(os.getcwd() + "/lights/competing_methods/MJLCMM.R")
trained_long_model, trained_mjlcmm = robjects.r["MJLCMM_fit"](data_R_train,
                                     robjects.StrVector(time_dep_feat),
                                     robjects.StrVector(time_indep_feat),
                                     alpha=2)
MJLCMM_pred = robjects.r["MJLCMM_score"](trained_long_model,
                                         trained_mjlcmm,
                                         time_indep_feat, data_R_test)
MJLCMM_marker = np.array(MJLCMM_pred.rx2('pprob')[2])
MJLCMM_c_index = c_index_score(T_test, MJLCMM_marker, delta_test)
MJLCMM_c_index = max(MJLCMM_c_index, 1 - MJLCMM_c_index)
MJLCMM_exe_time = time() - start

Be patient, multlcmm is running ... 
The program took 0.03 seconds 
The program took 70.39 seconds 
The program took 0.04 seconds 


In [7]:
# Multivariate shared random effect model.
start = time()
robjects.r.source(os.getcwd() + "/lights/competing_methods/JMBayes.R")
trained_JMBayes = robjects.r["fit"](data_R_train,
                                    robjects.StrVector(time_dep_feat),
                                    robjects.StrVector(time_indep_feat))
# JMBayes_pred = robjects.r["score"](trained_JMBayes, data_R_test, t_max=4)
# JMBayes_marker = np.array(JMBayes_pred.rx2('full.results')[0])
JMBayes_marker = np.array(robjects.r["score"](trained_JMBayes, data_R_test))
JMBayes_c_index = c_index_score(T_test, JMBayes_marker, delta_test)
JMBayes_c_index = max(JMBayes_c_index, 1 - JMBayes_c_index)
JMBayes_exe_time = time() - start

In [8]:
# lights
start = time()
fixed_effect_time_order = 1
learner = prox_QNMCEM(fixed_effect_time_order=fixed_effect_time_order,
                      max_iter=5, initialize=True, print_every=1,
                      compute_obj=True, simu=False,
                      asso_functions=["lp", "re"],
                      l_pen_SGL=0.02, eta_sp_gp_l1=.9, l_pen_EN=0.02)
learner.fit(X_lights_train, Y_lights_train, T_train, delta_train)
lights_marker = learner.predict_marker(X_lights_test, Y_lights_test)
lights_c_index = c_index_score(T_test, lights_marker, delta_test)
lights_c_index = max(lights_c_index, 1 - lights_c_index)
lights_exe_time = time() - start

Launching the solver prox_QNMCEM...
Launching the solver MLMM...
Launching the solver ULMM...


/Users/acsi/Working/lights/.env385/lib/python3.8/site-packages/statsmodels/regression/mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)
/Users/acsi/Working/lights/.env385/lib/python3.8/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


Done solving using ULMM in 3.29e-01 seconds
 n_iter  |   obj    | rel_obj 
       0 |  442.158 |      inf
       1 |  217.621 | 5.08e-01
       2 |  192.756 | 1.14e-01
       3 |  181.428 | 5.88e-02
       4 |  175.532 | 3.25e-02
       5 |  172.118 | 1.95e-02
Done solving using MLMM in 4.00e-01 seconds
 n_iter  |   obj    | rel_obj 
       0 |  40.3414 |      inf
       1 |  13.0348 | 6.77e-01
       2 |  9.65047 | 2.60e-01
       3 |  7.46459 | 2.27e-01
       4 |  6.94215 | 7.00e-02
       5 |  6.48506 | 6.58e-02
Done solving using prox_QNMCEM in 1.77e+01 seconds


In [9]:
t = PrettyTable(['Algos', 'C-Index', 'time'])
t.add_row(["Cox", "%g" % Cox_c_index, "%.3f" % Cox_exe_time])
t.add_row(["MJLCMM", "%g" % MJLCMM_c_index, "%.3f" % MJLCMM_exe_time])
t.add_row(["JMBayes", "%g" % JMBayes_c_index, "%.3f" % JMBayes_exe_time])
t.add_row(["lights", "%g" % lights_c_index, "%.3f" % lights_exe_time])
print(t)

+---------+----------+---------+
|  Algos  | C-Index  |   time  |
+---------+----------+---------+
|   Cox   | 0.998209 |  0.061  |
|  MJLCMM | 0.898836 |  70.518 |
| JMBayes | 0.77171  | 111.943 |
|  lights | 0.777977 |  19.281 |
+---------+----------+---------+
